In [4]:
exec(open('file_reader.py').read())

generating features..

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 581290271997968384

 581359544682614784

 758159624122097664

 763098277986209792

 764927075522260992

 767725956706414592

 768859780240773121

 769988636754505729

 774991078265094144

 775057555865206784

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 5812902719